# North American data Validation

## What is done 
1. Importing file from the folder
2. Doing some EDA
3. Renaming colums
4. Applying the validation rules
5. exporting the data

In [ ]:
# Enter the supplier name
file_name = input("Enter the source name: ").strip()

In [5]:
# working with DataFrames and Supporting array operations.
import pandas as pd
import numpy as np

# patterns matching
import glob
import re

# Handles file and directory operations
import os


# accessing to system-specific parameters and functions.
import sys

# Reads, writes, and modifies Excel files.
import openpyxl

# Handling file and directory operations like copying and moving files.
import shutil

# Creates a new Excel workbook and Opens an existing Excel file.
from openpyxl import Workbook
from openpyxl import load_workbook

# filling cells with colors or patterns.
from openpyxl.styles import PatternFill

# Converting a column index to an Excel-style column letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils.cell import get_column_letter

#  handling dates and times.
from datetime import datetime 

In [ ]:
# Define the folder path where the files are located
folder_path = r"\\Enter the path"

# Define file patterns
file_patterns = [f"{folder_path}/*.csv", f"{folder_path}/*.xlsx", f"{folder_path}/*.xls", f"{folder_path}/*.tsv"]

# Get all matching files
files = []
for pattern in file_patterns:
    files.extend(glob.glob(pattern))

# Iterate through each file 
for file_path in files:
    ext = file_path.split(".")[-1].lower()  # Get lowercase file extension

    if ext == "csv":
        df = pd.read_csv(file_path)
        print(f"Loaded CSV file: {file_path}")

    elif ext in ["xlsx", "xls"]:
        df = pd.read_excel(file_path)
        print(f"Loaded Excel file: {file_path}")

    elif ext == "tsv":
        df = pd.read_csv(file_path, sep="\t")
        print(f"Loaded TSV file: {file_path}") 

In [ ]:
# Reading the sheet for customer data
# df = pd.read_excel(file_path, sheet_name='Sheet1')

In [ ]:
# Check if the first row contains the word "Required"
if df.iloc[0].str.contains("Required").any():
    # Skip the first row
    df = df.iloc[1:, :].reset_index(drop=True)

In [ ]:
# check total rows and columns of the data
df.shape

In [ ]:
df.head()

In [ ]:
if any(re.search(r'country|country code', str(col), re.IGNORECASE) for col in df.columns):
    print("Country column is present")
else:
    print("Country column is NOT present")
    df["country code"] = "CA"

In [ ]:
# df.head(2)

In [ ]:
# drop all row with NaN
df = df.dropna(how='all')

In [ ]:
# Drop all cloumns with NaN
df.dropna(axis=1, how='all', inplace=True)

In [ ]:
# summery of the data
# df.info()

In [ ]:
# count of nulls in each columns
null_columns = df.isnull().sum()
null_columns = null_columns[null_columns > 0]
count_of_null = null_columns.reset_index()
count_of_null.columns = ['Column_Name', 'Null_Count']
count_of_null

In [ ]:
# Renaming Columns
def map_location_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Re-maps location-related column names in a DataFrame to standardized labels
    such as 'Address', 'City', 'ZipCode', 'Region', and 'Country'.
    """
    import unicodedata

    # Define a mapping dictionary based on substrings
    rename_map = {
        'address': 'Address',
        'street': 'Address',
        'city': 'City',
        'zip': 'ZipCode',
        'postal': 'ZipCode',
        'state': 'Region',
        'province': 'Region',
        'region': 'Region',
        'country': 'Country_code',
        'nation': 'Country'
    }

    def normalize_column(col):
        # Remove accents and lowercase
        col_norm = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode().lower()
        for keyword, standard in rename_map.items():
            if keyword in col_norm:
                return standard
        return col  # Return original if no match found

    # Apply the renaming
    new_columns = [normalize_column(col) for col in df.columns]
    df_renamed = df.copy()
    df_renamed.columns = new_columns

    return df_renamed

# Example usage:
# df_clean = map_location_columns(raw_df)
# print(df_clean.head())


In [ ]:
#clean the country_code
df1['Country_Code'] = df1['Country_Code'].astype(str).str.replace(' ', '')

In [ ]:
# clean post code
df1['Zip/Postal Code'] = df1['Zip/Postal Code'].astype(str).str.replace(r"[.,'!]", "", regex=True)

In [ ]:


def check_address_record(record, valid_postal_codes, region_aliases, postal_to_region_map):
    """
    Validates a single address record using several quality rules:
    - Country must be Canada (by name or code)
    - Postal code should be non-empty, valid, and format compliant
    - City and street must be sufficiently detailed
    - Region should be recognized and should align with the postal code
    """
    
    def is_valid_country(value):
        val = str(value).strip().lower()
        return val in {'ca', 'can', 'canada'}
    
    def is_valid_postcode(postal_code):
        if pd.isna(postal_code):
            return False
        code = str(postal_code).replace(" ", "").lower()
        return code in valid_postal_codes
    
    def is_format_compliant(postal_code):
        pattern = r'^[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d$'
        return bool(re.fullmatch(pattern, str(postal_code).strip()))
    
    def is_valid_city(city):
        return isinstance(city, str) and len(city.strip()) > 3

    def is_valid_street(address):
        return isinstance(address, str) and 3 < len(address.strip()) <= 100

    def is_recognized_region(region):
        return str(region).strip().lower() in region_aliases

    def region_matches_postal(postal_code, region):
        code = str(postal_code).replace(" ", "").lower()
        region_norm = str(region).strip().lower()
        expected = postal_to_region_map.get(code)
        return expected and (region_norm == expected or region_norm in region_aliases)

    return pd.Series([
        is_valid_country(record.get('Country')),
        is_valid_postcode(record.get('PostalCode')),
        is_valid_city(record.get('City')),
        is_valid_street(record.get('Street')),
        is_recognized_region(record.get('Region')),
        region_matches_postal(record.get('PostalCode'), record.get('Region')),
        is_format_compliant(record.get('PostalCode'))
    ])


In [ ]:
%%time

valid_postcodes_file = r'\\Enter the path'
postalcode_data = pd.read_csv(valid_postcodes_file)
# Normalize valid postcodes
valid_postcodes = postalcode_data['postalcode'] \
    .astype(str) \
    .str.replace(" ", "") \
    .str.lower() \
    .tolist()

In [ ]:
# Create a mapping from postal code to province
postcode_to_province_mapping = dict(zip(
    postalcode_data['postalcode'].str.replace(" ", "").str.lower(),
    postalcode_data['province'].str.strip().str.lower()
))

# Create province mapping (full name and short code)
province_mapping = set(
    postalcode_data['province'].str.strip().str.lower().tolist() + 
    postalcode_data['provincecode'].str.strip().str.lower().tolist()
)

In [ ]:
%%time
# Return the columns
df1[['CountryCodeValid', 
     'ZipPostalCodePopulated', 
     'PostalCodeInListValid', 
     'CityValid', 
     'StreetAddressValid', 
     'ProvinceValid', 
     'PostcodeMatchesProvince', 
     'PostalCodeFormatValid']] = df1.apply(
    lambda row: validate_record(row, 
                                valid_postcodes=valid_postcodes, 
                                province_mapping=province_mapping, 
                                postcode_to_province_mapping=postcode_to_province_mapping), 
    axis=1
)

In [ ]:
# Add an bool column PassFail
df1['PassFail'] = df1[['CountryCodeValid', 'ZipPostalCodePopulated', 'PostalCodeInListValid', 
                       'CityValid', 'StreetAddressValid','PostcodeMatchesProvince','PostalCodeFormatValid']].all(axis=1).map({True: 'Pass', False: 'Fail'})

In [ ]:
# checking the postal code format
df1['PostalCodeStatus'] = np.where(
    (df1['PostalCodeFormatValid'] == True) & (df1['PostalCodeInListValid'] == True), "Valid",
    np.where(
        (df1['PostalCodeFormatValid'] == True) & (df1['PostalCodeInListValid'] == False), "Could be valid please check",
        "Not Valid"
    )
)

In [ ]:
# Drop the ProvienceValid Column
df_result = df1.drop(columns=['ProvinceValid'])

In [ ]:
# df_result.head()

In [ ]:
%%time
# Placing the generated file in the folder
current_date = datetime.now().strftime('%Y%m%d')
output_file_path = rf'\\path\CA_validation_OUT\{file_name}_validation__{current_date}.xlsx'
df_result.to_excel(output_file_path, index=False)

In [ ]:
%%time
# Define styles
valid_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")  # Green
invalid_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")  # Red
pass_fill = PatternFill(start_color="66CDAA", end_color="66CDAA", fill_type="solid")  # Medium Green
fail_fill = PatternFill(start_color="CD5C5C", end_color="CD5C5C", fill_type="solid")  # Dark Red

# Define styles for PostalCodeStatus
could_be_valid_soft_blue = PatternFill(start_color="A9D0F5", end_color="A9D0F5", fill_type="solid")  # Light Blue
not_valid_light_orange = PatternFill(start_color="FFDAB9", end_color="FFDAB9", fill_type="solid")  # Light Orange
valid_soft_green = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")  # Green


# Load workbook
wb = openpyxl.load_workbook(output_file_path)
ws = wb.active

# Map headers to columns in the worksheet
headers = {cell.value: cell.column for cell in ws[1]}  # Assumes the first row is the header

# Columns to format
columns_to_format = ['CountryCodeValid', 'ZipPostalCodePopulated', 'PostalCodeInListValid', 
                     'CityValid', 'StreetAddressValid', 'PostcodeMatchesProvince', 'PostalCodeFormatValid']
pass_fail_col = 'PassFail'
postalcodestats_col = 'PostalCodeStatus'  # Column for PostalCodeStatus formatting

# Check if all required columns exist in the worksheet
for col in columns_to_format + [pass_fail_col, postalcodestats_col]:
    if col not in headers:
        raise ValueError(f"Column '{col}' not found in worksheet headers")

# Iterate over rows and apply formatting
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):  # Skip header row
    # Apply formatting to the columns in bulk
    for col in columns_to_format:
        cell = row[headers[col] - 1]  # Adjust for 0-based index
        if cell.value:  # Assume valid if True or non-empty
            cell.fill = valid_fill
        else:
            cell.fill = invalid_fill

    # Format PassFail column
    pass_fail_cell = row[headers[pass_fail_col] - 1]  # Adjust for 0-based index
    if pass_fail_cell.value == 'Pass':
        pass_fail_cell.fill = pass_fill
    else:
        pass_fail_cell.fill = fail_fill

    # Apply color to PostalCodeStatus column
    postalcodestats_cell = row[headers[postalcodestats_col] - 1]
    if postalcodestats_cell.value == "Valid":
        postalcodestats_cell.fill = valid_soft_green
    elif postalcodestats_cell.value == "Not Valid":
        postalcodestats_cell.fill = not_valid_light_orange
    elif postalcodestats_cell.value == "Could be valid please check":
        postalcodestats_cell.fill = could_be_valid_soft_blue

# Save workbook after all changes
wb.save(output_file_path)